Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/home/adriann/Lambda/DS-Unit-2-Applied-Modeling/module2-wrangle-ml-datasets/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

order_products__train.csv
aisles.csv
products.csv
order_products__prior.csv
departments.csv
orders.csv


### For each csv file, look at its shape & head 

In [11]:
import pandas as pd
from IPython.display import display

def preview():
    for filename in glob('*.csv'):
        df = pd.read_csv(filename)
        print('\n', filename, df.shape)
        display(df.head())
        
preview()


 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 products.csv (49688, 4)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [13]:
order_products__train = pd.read_csv('order_products__train.csv')

In [23]:
temp = order_products__train.sort_values(by='product_id', ascending=False).head()
temp

,order_id,product_id,add_to_cart_order,reordered
891142,2198380,49688,10,0
1209047,2986153,49688,16,0
269349,655800,49688,10,1
1014644,2508423,49688,3,0
446055,1092104,49687,1,0


In [22]:
temp.groupby('product_id').count()

,order_id,add_to_cart_order,reordered
product_id,,,
49687,1,1,1
49688,4,4,4


In [16]:
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [27]:
order_products__train.groupby('product_id').order_id.count().sort_values(ascending=False)

product_id
24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
29560        1
8001         1
29566        1
8005         1
24836        1
Name: order_id, Length: 39123, dtype: int64

In [33]:
# Read the products table
# Figure out the product name for product id 24852
products = pd.read_csv('products.csv')
products[products['product_id'] == 24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [37]:
# Not specifying on defaults to whatever both dataframes have in common
train = pd.merge(order_products__train, products, how='inner')

In [38]:
train['product_name'].value_counts()

Banana                                                                      18726
Bag of Organic Bananas                                                      15480
Organic Strawberries                                                        10894
Organic Baby Spinach                                                         9784
Large Lemon                                                                  8135
                                                                            ...  
Tikka Masala Curry Sauce                                                        1
Peach Multigrain Cereal with Blackberries Organic Baby Food for 3 and Up        1
Cold Remedy Plus Natural Immune Support Orange QuickMelts                       1
Chicken Condensed Consomme                                                      1
Blood Orange Cocktail Mixer                                                     1
Name: product_name, Length: 39123, dtype: int64

## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [42]:
train['bananas'] = train['product_name'] == 'Banana'
train

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
0,1,49302,1,1,Bulgarian Yogurt,120,16,False
1,816049,49302,7,1,Bulgarian Yogurt,120,16,False
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,False
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,False
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,False
...,...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,Sprinkles Decors,97,13,False
1384613,3420084,47935,20,0,Classic Original Lip Balm SPF 12,73,11,False
1384614,3420084,9491,21,0,Goats Milk & Chai Soap,25,11,False
1384615,3420088,16380,12,0,Stevia Sweetener,97,13,False


In [45]:
# 1.4% of the products ordered are (non-organic) bananas
train['bananas'].value_counts(normalize=True)

False    0.986476
True     0.013524
Name: bananas, dtype: float64

In [52]:
# For each order, were any of the products bananas?
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()

In [55]:
# ~14% of orders included bananas
train_wrangled['bananas'].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

## Which customers have ordered bananas before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [63]:
# In the train table, which orders included bananas?
banana_order_id = train[train['bananas']==True].order_id

In [60]:
# Look at the orders table
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2640906,2814272,158984,prior,1,2,10,NaN
3094898,1815584,186623,prior,30,1,8,6.0
737723,830616,44415,prior,27,3,15,8.0
3231277,3213264,194792,prior,1,2,10,NaN
2481057,187884,149334,prior,11,2,18,30.0


In [64]:
# In the orders table, which orders included bananas?
orders[orders.order_id.isin(banana_order_id)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
25,1492625,2,train,15,1,11,30.0
485,698604,34,train,6,4,13,30.0
623,1864787,43,train,12,5,11,26.0
652,1947848,46,train,20,4,20,11.0
881,13318,56,train,13,1,9,6.0
...,...,...,...,...,...,...,...
3420581,681092,206181,train,15,1,13,30.0
3420692,2530125,206191,train,6,0,9,30.0
3420786,3383615,206196,train,5,0,15,30.0
3420933,1716008,206205,train,4,1,16,10.0


In [66]:
# Check an order_id from above, confirm that yes it includes bananas
train[train.order_id == 1492625]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,bananas
40632,1492625,11913,28,0,Shelled Pistachios,117,19,False
137883,1492625,24852,8,1,Banana,24,4,True
199519,1492625,9387,10,0,Granny Smith Apples,24,4,False
202785,1492625,45066,9,1,Honeycrisp Apple,24,4,False
241750,1492625,5450,11,1,Small Hass Avocado,24,4,False
269520,1492625,33957,21,1,Mexican Casserole Bowl,38,1,False
286130,1492625,41787,5,1,Bartlett Pears,24,4,False
356655,1492625,22963,1,1,Organic Roasted Turkey Breast,96,20,False
503295,1492625,48523,25,0,Birthday Cake Light Ice Cream,37,1,False
523860,1492625,38547,13,0,Bubblegum Flavor Natural Chewing Gum,46,19,False


In [67]:
banana_orders = orders[orders.order_id.isin(banana_order_id)]

In [69]:
# In the orders table, which users have bought bananas?
banana_orders.user_id.unique()

array([     2,     34,     43, ..., 206196, 206205, 206209])

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [70]:
# orders table, shape before getting subset
orders.shape

(3421083, 7)

In [72]:
# orders table, shape after getting subset
orders_subset = orders[orders.user_id.isin(banana_order_id)]
orders_subset.shape

(18425, 7)

In [76]:
orders_subset

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
3572,2787288,226,prior,1,5,19,NaN
3573,3415661,226,prior,2,4,11,6.0
3574,1614014,226,prior,3,5,8,8.0
3575,2621348,226,prior,4,4,7,6.0
3576,1276412,226,prior,5,2,20,5.0
...,...,...,...,...,...,...,...
3416366,724254,205908,prior,72,3,10,5.0
3416367,561145,205908,prior,73,0,1,4.0
3416368,721897,205908,prior,74,3,15,3.0
3416369,938870,205908,prior,75,1,13,5.0


## What features can we engineer? We want to predict, will these customers reorder bananas on their next order?

### Brain storming:
- Products per order
- Time of day
- Have they reordered banans before? (Have ordered bananas >= 2 times)
- Other fruit they buy
- size of orders (customers with smaller orders on average are less likely to be reordering any perticular product on 
- Frequency of banana orders: 
    - % of orders
    - Time between banana orders: every n days on average
    - Raw count: Total orders, How many times have you ordered bananas
- Recency of banana orders
    - n days since you ordered bananas

In [75]:
# Pick out an individual user for esxploration and experiments
USER_ID = 2
orders_subset[orders_subset.user_id==USER_ID]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
